In [49]:
# import package
# installed via pip
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
from emtracks.interpolations import *
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm

mao13mapdatadir = '/home/shared_data/Bmaps/Mau13/subtracted/'
mao13mapsavedir = '/home/shared_data/mao10,mao13_analysis/data/mao13contourplots4/'

In [7]:
def get_uniform_phi(N):
    phis = np.linspace(0, 2*math.pi, N)
    return phis

def get_uniform_theta(N):
    thetas = np.linspace(2.085, 2.24, N)
    return thetas

files = sorted(os.listdir(mao13mapdatadir))

In [22]:
files[0][6:13]


'0.10xDS'

In [24]:
files_new = []
for file in files:
    if file[6:13] == '1.00xDS':
        files_new.append(file)
files = files_new
files

['Mau13_1.00xDS_0.00xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.10xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.20xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.30xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.40xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.50xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.60xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.70xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.80xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.90xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.91xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.92xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.93xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.94xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.95xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.96xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.97xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.98xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.99xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.00xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.01xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.02xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.03xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.04xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.05xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.06xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.07xPS-TS_DSMap.p',
 

In [25]:
def getDSfield(file):
    return file.split('_')[1].split('x')[0]
def getPSfield(file):
    return file.split('_')[2].split('x')[0]

def getfiles(files, PSfields, DSfields):
    PSfieldsrounded = [round(num, 3) for num in PSfields]
    DSfieldsrounded = [round(num, 3) for num in DSfields]
    
    filedata = []
    for file in files:
        if float(getDSfield(file)) in DSfieldsrounded:
            if float(getPSfield(file)) in PSfieldsrounded:
                filedata.append(file)
    
    return filedata

filedata = getfiles(files, np.linspace(0.90, 1.10, 21), np.array([1.]))

In [28]:
filedata

['Mau13_1.00xDS_0.90xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.91xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.92xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.93xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.94xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.95xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.96xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.97xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.98xPS-TS_DSMap.p',
 'Mau13_1.00xDS_0.99xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.00xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.01xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.02xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.03xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.04xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.05xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.06xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.07xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.08xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.09xPS-TS_DSMap.p',
 'Mau13_1.00xDS_1.10xPS-TS_DSMap.p']

In [41]:
def get_funcs(files): 
    data = {}
    for file in files:
        DS = getDSfield(file)
        PS = getPSfield(file)
        data.update({PS:get_df_interp_func(df = pd.read_pickle(mao13mapdatadir+'/'+file), gauss=False)})
#         with open(mao13mapdatadir+'/'+file, 'rb') as pickle_file:
#             data.update({PS:get_df_interp_func(df = pkl.load(pickle_file), gauss=False)})
    return data

In [42]:
data = get_funcs(filedata)
data

{'0.90': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.91': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.92': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.93': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.94': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.95': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.96': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.97': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.98': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '0.99': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '1.00': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '1.01': <function emtracks.mapinterp.get_df_interp_func.<locals>.interp(p_vec)>,
 '1.02': <functi

In [45]:
def run_solver(phi, theta, field, fielddescrip, N_calc, xnaught, ynaught, znaught):
    ic_Mu2e = InitConds(t0=0., tf=2e-7, N_t=N_calc,
                    x0=xnaught, y0=ynaught, z0=znaught,
                    p0=104.96, theta0=theta, phi0=phi)
    e_solver = trajectory_solver(ic_Mu2e, B_func=field, bounds=bounds_Mu2e)
    sol = e_solver.solve_trajectory(verbose = False, atol=1e-10, rtol=1e-10)
    e_solver.dataframe['r'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    e_solver.dataframe['rfromcenter'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    filename = f'{fielddescrip}_{ic_Mu2e.theta0:0.3f}_{ic_Mu2e.phi0:0.3f}_{ic_Mu2e.x0:0.3f}_.pkl'
    e_solver.to_pickle(mao13mapsavedir+filename)   

In [46]:
def runbothsolvers(phi, theta, fielddata, N_calc, xnaught, ynaught, znaught):
    keys = list(fielddata.keys())
    values = list(fielddata.values())
        
    for i in range(0, len(keys), 1):
        run_solver(phi, theta, values[i], str(keys[i]), N_calc, xnaught, ynaught, znaught)

In [47]:
def savepickle(N, files, ncalc = 10001):
    phis = get_uniform_phi(N)
    thetas = get_uniform_theta(N)
    data = get_funcs(files)
    
    numcpu = multiprocessing.cpu_count()
    Parallel(n_jobs = numcpu)(delayed(runbothsolvers)(phi, theta, data, ncalc, 0.054094482, 0.03873037, 5.988900879) for theta in tqdm(thetas, desc = 'theta') for phi in phis)


In [51]:
savepickle(50, filedata)

In [ ]:
save plots